<a href="https://colab.research.google.com/github/lavlugit/lavlugit/blob/main/Open_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =============================
# Step 1: Install dependencies
# =============================
#!pip install openai scholarly fpdf gradio

# =============================
# Step 2: Import libraries
# =============================
import os
from fpdf import FPDF
from scholarly import scholarly
import gradio as gr
from openai import OpenAI

# =============================
# Step 3: OpenAI API Key
# =============================
# In Colab, set your key like this before running:
# %env OPENAI_API_KEY=your_api_key_here
import os
from getpass import getpass
from openai import OpenAI

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    api_key = getpass("Enter your OpenAI API key: ")

client = OpenAI(api_key=api_key)


# =============================
# Step 4: Functions
# =============================
def generate_section(section_name, topic, keywords=""):
    """Generate one section of the paper using OpenAI API."""
    prompt = f"""
    Generate a research paper section:
    Section: {section_name}
    Topic: {topic}
    Keywords: {keywords}
    Write in a formal academic style. Be concise, structured, and coherent.
    """
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",   # or "gpt-4o-mini" if you have access
        messages=[{"role": "user", "content": prompt}],
        max_tokens=400,
        temperature=0.7
    )
    return response.choices[0].message.content.strip()


def fetch_references(topic, num=5):
    """Fetch references using scholarly library."""
    search_query = scholarly.search_pubs(topic)
    refs = []
    try:
        for i in range(num):
            pub = next(search_query)
            title = pub['bib'].get('title', 'N/A')
            author = pub['bib'].get('author', 'N/A')
            year = pub['bib'].get('year', 'N/A')
            journal = pub['bib'].get('journal', '')
            refs.append(f"{author} ({year}). {title}. {journal}")
    except Exception:
        pass
    return refs if refs else ["No references found."]


def generate_paper(topic, keywords):
    """Generate full paper with sections + PDF output."""
    sections = ["Title", "Abstract", "Introduction", "Methodology",
                "Results", "Discussion", "Conclusion", "References"]
    paper = {}

    for sec in sections:
        if sec.lower() != "references":
            paper[sec] = generate_section(sec, topic, keywords)
        else:
            refs = fetch_references(topic, num=5)
            paper[sec] = "\n".join(refs)

    # PDF builder

    # Helper to avoid Unicode issues


    class PDF(FPDF):
        def header(self):
            self.set_font('Arial', 'B', 12)
            self.cell(0, 10, f"{topic}", 0, 1, 'C')
            self.ln(5)
        def footer(self):
            self.set_y(-15)
            self.set_font('Arial', 'I', 10)
            self.cell(0, 10, f'Page {self.page_no()}', 0, 0, 'C')

    pdf = PDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()

    # Table of Contents
    pdf.set_font("Arial", 'B', 16)
    pdf.multi_cell(0, 10, "Table of Contents\n")
    pdf.set_font("Arial", '', 12)
    for i, sec in enumerate(sections, 1):
        pdf.multi_cell(0, 8, f"{i}. {sec}")
    pdf.add_page()

    # Add sections
    # Helper to avoid Unicode issues
    def safe_text(text):
       return text.encode('latin-1', 'replace').decode('latin-1')

  # Add sections
    for sec in sections:
      pdf.set_font("Arial", 'B', 14)
      pdf.multi_cell(0, 10, safe_text(f"{sec}\n"))
      pdf.set_font("Arial", '', 12)
      pdf.multi_cell(0, 8, safe_text(paper[sec]))
      pdf.add_page()


    pdf_file_name = f"{topic.replace(' ', '_')}_AI_Paper.pdf"
    pdf.output(pdf_file_name)

    return pdf_file_name

# =============================
# Step 5: Gradio Interface
# =============================
iface = gr.Interface(
    fn=generate_paper,
    inputs=[
        gr.Textbox(label="Research Paper Topic"),
        gr.Textbox(label="Keywords (comma separated)")
    ],
    outputs=gr.File(label="Download PDF", file_types=[".pdf"]),
    title="AI Research Paper Generator",
    description="Enter a topic and keywords to generate a full AI-powered research paper PDF with references."
)

iface.launch(share=True, debug=True)

